(comp/00-algorithms)=
# Coding Problems

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=brightgreen)
[![Source](https://img.shields.io/static/v1.svg?label=GitHub&message=Source&color=181717&logo=GitHub)](https://github.com/particle1331/ok-transformer/blob/master/docs/nb/comp/00-algorithms.ipynb)
[![Stars](https://img.shields.io/github/stars/particle1331/ok-transformer?style=social)](https://github.com/particle1331/ok-transformer)

---

**References:** {cite}`durr2020competitive` {cite}`Laaksonen20`

## Backtracking

### Reverse Shuffle Merge

**Problem.** [[Reverse Shuffle Merge](https://www.hackerrank.com/challenges/reverse-shuffle-merge/problem) (HR)]. Given a string $S$ containing characters with even counts, we want to find a lexically minimal string $A$ containing half of each characters of $S$ such that $a_i = s_{j_i}$ where $j_{k} \leq j_{k-1} \leq \ldots \leq j_1$ with $k = \frac{1}{2}|S|.$ For example, if $S$ is `eggcegcg` then $A$ is the string `cegg` which can be constructed as `·gg·e·c·` within $S$.

**Solution.** Note that we can iterate over $S$ from right to left choosing to skip or take a character. Since $A$ has to be minimal, we want to take minimal characters, and skip those that are large. However, skipping too often can also result in a suboptimal solution. Consider `S = 'cdeedcaaaa'` here skipping `c` and `d` gets us in trouble since `e` reaches its skip limit of 1/2 its count, and we are forced to take it, getting the suboptimal solution `A = 'aaedc'`. Hence, we want to take characters, unless we are forced to skip, or we have better options:

In [1]:
from collections import Counter

def solve(S):
    counts = Counter(S)
    chars = counts.keys()
    A = ['.']
    take_count = {c: 0 for c in chars}
    skip_count = {c: 0 for c in chars}

    def take(s):
        A.append(s)
        take_count[s] += 1

    def skip(s):
        skip_count[s] += 1

    def squash(s):
        while (len(A) > 1) and (A[-1] > s) and (skip_count[A[-1]] < counts[A[-1]] / 2):
            skip_count[A[-1]] += 1
            take_count[A[-1]] -= 1
            A.pop()
        take(s)

    for s in S[::-1]:        
        if take_count[s] == counts[s] / 2:
            skip(s)
        else:
            squash(s)

    return "".join(A[1:])


S = "eggcegcg"
A = solve(S)
A

'cegg'

Notice that instead of `take(s)` we have `squash(s)`. The trick is to construct chains: `a <= a <= c <= d` and  if we encounter `b` such that `d > b`, then we backtrack by popping `d` and `c` assuming the skip limit allows it. Our sequence then becomes `a <= a <= b`. The equals is important here to allow consecutive same characters. This is also why we added `.` at the start which is smaller than any character. Observe that the `squash` function only takes into account `A` and ignores skipped characters. This is fine since instances of these characters are already optimally placed in the chain. It only adds more skipped characters which are suboptimal in the chain compared to the current queried character.

Trying a complicated input:

In [2]:
def viz(S):
    chars = Counter(S)
    d = {c: [] for c in sorted(chars.keys(), reverse=True)}

    for s in S[::-1]:
        for c in d.keys():
            if c == s:
                d[c].append(s + ' ')
            else:
                d[c].append('  ')

    for c in d.keys():
        print(''.join(d[c]))



S = "dcadeebcccaaba."
viz(S)
print()
print(f"S = {S[:-1]}")
print("A =", solve(S)[1:])

                  e e         
                      d     d 
          c c c           c   
    b           b             
  a   a a               a     
.                             

S = dcadeebcccaaba
A = aacbecd


```{figure} ../../img/greedy-1-soln-1.png
---
width: 60%
---
To respect optimality, we try as much as possible to have our curve in the bottom half of the diagonal. At the start, `b` is squashed after getting `a <= b ? a` while the third `a` skipped. Then, we get consecutive `c <= c ? b` (last `c` is skipped due to take limit) only the latter `c` is popped due to skip limit. Next, we encounter two `e`, the latter skipped due to take limit. And so on. Marked characters are skipped while unmarked characters were squashed (essentially also skipped).
```